# MIMIC Medication Flow
This notebook looks to answer questions on the consistency and flow of medication throughout the medication tables. The tables to be explored are:
- *prescriptions*
- *pharmacy*
- *emar*
- *emar_detail*

The statistics calculated throughout the notebook will be based on the following criteria:
- anchor_year_group = 2017-2019
- at least on emar event present in *emar*

The primary questions to answer are:
- How many emar events have matching pharmacy_id with *pharmacy*
- How many emar events have matching pharmacy_id with *prescriptions*
- How many *prescriptions* pharmacy_id match with *pharmacy* pharmacy_id
- For matching pharmacy_id, do the medication labels stay consistent from *prescriptions* to *pharmacy* to *emar*


In [ ]:
# Imports
import sys
import pandas as pd
import numpy as np
import psycopg2
from pathlib import Path
import json
from uuid import uuid5, NAMESPACE_OID
import datetime
from dotenv import load_dotenv

In [ ]:
# Load environment variables
load_dotenv(load_dotenv(Path(Path.cwd()).parents[0] / '.env'))

SQLUSER = os.getenv('SQLUSER')
SQLPASS = os.getenv('SQLPASS')
DBNAME_MIMIC = os.getenv('DBNAME_MIMIC')
HOST = os.getenv('DBHOST')

db_conn = psycopg2.connect(dbname=DBNAME_MIMIC, user=SQLUSER, password=SQLPASS, host=HOST)

## Medication Cohort Stats

In [ ]:
q_med_cohort = """
    SELECT * FROM mimic_core.patients p
    WHERE anchor_year_group = '2017 - 2019'
"""
med_cohort = pd.read_sql_query(q_med_cohort, db_conn)

In [ ]:
q_emar_patients = """
    SELECT DISTINCT em.subject_id
    FROM 
        mimic_core.patients p 
        LEFT JOIN mimic_hosp.emar em
            ON p.subject_id = em.subject_id
    WHERE anchor_year_group = '2017 - 2019'
"""
emar_patients = pd.read_sql_query(q_emar_patients, db_conn)

In [ ]:
print(f'2017-2019 anchor group has {len(med_cohort):,} patients, but only {len(emar_patients):,} patients have emar records.')
print(f'Thus, of the cohort, {len(emar_patients)/len(med_cohort):.0%} of patients have an emar event')

## Emar connections
- How many emar events have matching pharmacy_id with *pharmacy*
- How many emar events have matching pharmacy_id with *prescriptions*

In [ ]:
q_emar_events = """
    SELECT em.subject_id, em.hadm_id, em.pharmacy_id, em.medication
    FROM 
        mimic_hosp.emar em
        INNER JOIN mimic_core.patients p 
            ON em.subject_id = p.subject_id
    WHERE anchor_year_group = '2017 - 2019'
"""
emar_events = pd.read_sql_query(q_emar_events, db_conn)

In [ ]:
pharmacy_id_missing = emar_events.pharmacy_id.isna().sum()
total_emar_events = len(emar_events)
percent_pharmacy_missing = pharmacy_id_missing/total_emar_events
distinct_emar_phid = len(emar_events.pharmacy_id.unique())
print(f'Of the {total_emar_events:,} emar events, {percent_pharmacy_missing:.0%} have a null pharmacy_id.')
print(f'For all the emar events there are {distinct_emar_phid:,} distinct pharmacy_id')

In [ ]:
q_pharmacy = """
    SELECT ph.subject_id, ph.hadm_id, ph.pharmacy_id, ph.medication
    FROM 
        mimic_hosp.pharmacy ph
        INNER JOIN mimic_core.patients pat 
            ON ph.subject_id = pat.subject_id
    WHERE anchor_year_group = '2017 - 2019'
"""
df_pharmacy = pd.read_sql_query(q_pharmacy, db_conn)

In [ ]:
print(f'For all the pharmacy events there are {len(df_pharmacy):,} distinct pharmacy_id')

In [ ]:
q_prescriptions = """
    SELECT pr.subject_id, pr.hadm_id, pr.pharmacy_id, pr.drug, pr.drug_type
    FROM 
        mimic_hosp.prescriptions pr
        INNER JOIN mimic_core.patients pat 
            ON pr.subject_id = pat.subject_id
    WHERE anchor_year_group = '2017 - 2019'
"""
df_prescriptions = pd.read_sql_query(q_prescriptions, db_conn)

In [ ]:
distinct_prescription_pharmacy_id = len(df_prescriptions.pharmacy_id.unique())
print(f'For all the prescriptions events there are {distinct_prescription_pharmacy_id:,} distinct pharmacy_id')

In [ ]:
df_phid_emar = pd.DataFrame(emar_events.pharmacy_id.unique())
df_phid_pharmacy = pd.DataFrame(df_pharmacy.pharmacy_id.unique())
df_phid_prescriptions = pd.DataFrame(df_prescriptions.pharmacy_id.unique())

In [ ]:
len(df_phid_emar)

In [ ]:
emar_phid_in_pharmacy = len(df_phid_emar.merge(df_phid_pharmacy, how='inner'))
emar_phid_in_prescriptions = len(df_phid_emar.merge(df_phid_prescriptions, how='inner'))
pharmacy_phid_in_prescriptions = len(df_phid_pharmacy.merge(df_phid_prescriptions, how='inner'))
distinct_emar_phid


In [ ]:
print(f'Of the {distinct_emar_phid:,} distinct pharmacy_id in emar, {emar_phid_in_pharmacy:,} exist in pharmacy, and {emar_phid_in_prescriptions:,} in prescriptions')
print(f'That works out to {emar_phid_in_pharmacy/distinct_emar_phid:.0%} in pharmacy and {emar_phid_in_prescriptions/distinct_emar_phid:.0%} in prescriptions')
print(f'Also there are {pharmacy_phid_in_prescriptions/len(df_phid_pharmacy):.2%} prescriptions pharmacy_id that show up in pharmacy')
print(f'Also there are {pharmacy_phid_in_prescriptions/len(df_phid_prescriptions):.2%} pharmacy pharmacy_id that show up in prescriptions')

## Pharmacy and prescriptions medication overlap
Check to see if the medication and drug name match for the same pharmacy_id. Prescriptions may have multiple meds but just use main for comparison

In [ ]:
distinct_prescriptions = len(df_prescriptions.pharmacy_id.unique())
medication_mix = df_prescriptions.groupby(['pharmacy_id']).filter(lambda x: len(x) > 1)


In [ ]:
num_medication_mix = len(medication_mix)
print(f'There are {distinct_prescriptions:,} distinct pharmacy_id in prescriptions. Of those, {num_medication_mix:,} are a combination of multiple medication in one prescription order.')

In [ ]:
len(df_prescriptions)

#### Check overlap of pharmacy_id beteween pharmacy and prescriptions

In [ ]:
pharmacy_id_overlap = df_phid_pharmacy.merge(df_phid_prescriptions, how='inner')

In [ ]:
print(f'Distinct pharmacy_id in pharmacy {len(df_phid_pharmacy):,}')
print(f'Distinct pharmacy_id in prescriptions {len(df_phid_prescriptions):,}')
print(f'Distinct pharmacy_id overlapping pharmacy and prescriptions {len(pharmacy_id_overlap):,}')

In this medication cohort, the prescriptions pharmacy_id are fully captured in the pharmacy table. 

Now let's check if the medication name matches across the two tables.

In [ ]:
df_main_meds = df_prescriptions.loc[df_prescriptions['drug_type'] == 'MAIN']
df_merged_meds = df_main_meds.merge(df_pharmacy, how='left', on=['subject_id', 'hadm_id', 'pharmacy_id'])
med_compare = df_merged_meds.loc[df_merged_meds['drug'] != df_merged_meds['medication']]

In [ ]:
print(f'There are {len(df_merged_meds):,} medication entries in the cohort')
print(f'There are {len(med_compare):,} medication mismatches between pharmacy and prescriptions')
print(f'That amounts to {len(med_compare)/len(df_merged_meds):.2%} of medication mismatching')

In [ ]:
# some example mismatches
med_compare.iloc[0:50]

From the medication comparison between pharmacy and prescriptions, the majority of mismatches appear to be more detail added in pharmacy medication. Interestingly there are empt pharmacy.medication values and some medication names set to numeric values (ie 1,2)

In [ ]:
len(med_compare)

In [ ]:
len(emar_events)

In [ ]:
emar_events.pharmacy_id.isna().sum()

In [ ]:
len(emar_events.loc[emar_events['pharmacy_id'].isna() == False])

Check medication matching between emar and pharmacy/prescriptions

In [ ]:
df_main_meds = df_prescriptions.loc[df_prescriptions['drug_type'] == 'MAIN']

# grab only meds that have pharmacy_id to match against pharmacy. Rename medication to not overlap pharmacy.medication
df_emar_meds = emar_events.loc[emar_events['pharmacy_id'].isna() == False].rename(columns={'medication': 'emar_medication'}) 
df_merged_emar_and_pharma = df_pharmacy.merge(df_emar_meds, how='inner', on=['subject_id', 'hadm_id', 'pharmacy_id']) # only grab pharmacy_id that are in both
df_merged_emar_and_presc = df_main_meds.merge(df_emar_meds, how='inner', on=['subject_id', 'hadm_id', 'pharmacy_id']) # only grab pharmacy id that are in both
emar_compare_pharma = df_merged_emar_and_pharma.loc[df_merged_emar_and_pharma['medication'] != df_merged_emar_and_pharma['emar_medication']]
emar_compare_pharma = emar_compare_pharma[emar_compare_pharma['medication'].notnull() | emar_compare_pharma['emar_medication'].notnull()] 

emar_compare_presc = df_merged_emar_and_presc.loc[df_merged_emar_and_presc['drug'] != df_merged_emar_and_presc['emar_medication']]

In [ ]:
print(f'There are {len(df_emar_meds):,} emar medication in the cohort')
print(f'There are {len(emar_compare_pharma):,} medication mismatches between emar and pharmacy')
print(f'There are {len(emar_compare_presc):,} medication mismatches between emar and prescriptions')
print(f'That amounts to {len(emar_compare_pharma)/len(df_merged_emar_and_pharma):.2%} of medication mismatching between emar/pharmacy')
print(f'That amounts to {len(emar_compare_presc)/len(df_merged_emar_and_presc):.2%} of medication mismatching between emar/prescriptions')

In [ ]:
emar_compare_pharma
df_merged_emar_and_pharma.iloc[7].medication == df_merged_emar_and_pharma.iloc[7].emar_medication

In [ ]:
emar_compare_presc.iloc[100:150]

In [ ]:
emar_compare_pharma[emar_compare_pharma['medication'].notnull() | emar_compare_pharma['emar_medication'].notnull()] 

## Emar stats alone

In [ ]:
q_emar_detail_events = """
    SELECT emd.subject_id, em.pharmacy_id, em.medication, emd.product_code, em.poe_id
    FROM 
        mimic_hosp.emar_detail emd
        LEFT JOIN mimic_hosp.emar em
            ON emd.emar_id = em.emar_id
        INNER JOIN mimic_core.patients p 
            ON em.subject_id = p.subject_id
    WHERE 
        anchor_year_group = '2017 - 2019'
        AND emd.parent_field_ordinal IS NOT NULL;
"""
emar_detail_events = pd.read_sql_query(q_emar_detail_events, db_conn)

In [ ]:

sys.getsizeof(emar_detail_events) # 2GB!

In [ ]:
null_product_code = emar_detail_events.product_code.isna().sum()
null_medication = emar_detail_events.medication.isna().sum()
null_poe_id = emar_detail_events.poe_id.isna().sum()
null_pharmacy_id = emar_detail_events.pharmacy_id.isna().sum()
total_emar_detail_size = len(emar_detail_events)

print(f'There are {null_product_code/total_emar_detail_size:.2%} null product codes')
print(f'There are {null_medication/total_emar_detail_size:.2%} null medication codes')
print(f'There are {null_poe_id/total_emar_detail_size:.2%} null poe_id')
print(f'There are {null_pharmacy_id/total_emar_detail_size:.2%} null pharmacy_id')


Order of medication in emar:
- emar_detail.product_code (more specific)
- emar.medication
- poe.order_type (based on poe_id) --> mostly TPN/IV events

In [ ]:
print(f'{1-null_product_code/total_emar_detail_size:.2%} emar events that depend on product_code')
print(f'The remaining {null_product_code/total_emar_detail_size:.2%} emar events then try to use medication')

In [ ]:
em_medication_no_product_code = (emar_detail_events.product_code.isna() & emar_detail_events.medication.notna()).sum()

In [ ]:
remaining_for_poe = (1-null_product_code/total_emar_detail_size)+em_medication_no_product_code/total_emar_detail_size
print(f'{em_medication_no_product_code/total_emar_detail_size:.2%} emar events that depend on medication when no product_code')
print(f'The remaining {1-remaining_for_poe:.2%} emar events then use poe_id')

In [ ]:
total_emar_detail_size

In [ ]:
print(f'{1-null_medication/total_emar_detail_size:.2%} emar events that depend on medication')
print(f'The remaining {null_medication/total_emar_detail_size:.2%} emar events then try to use product_code')

In [ ]:
em_product_code_no_medication = (emar_detail_events.product_code.notna() & emar_detail_events.medication.isna()).sum()

In [ ]:
remaining_for_poe2 = (1-null_medication/total_emar_detail_size)+em_product_code_no_medication/total_emar_detail_size
print(f'{em_product_code_no_medication/total_emar_detail_size:.2%} emar events that depend on product_code when no medication')
print(f'The remaining {1-remaining_for_poe2:.2%} emar events then use poe_id')

## Exploring Prescriptions

In [ ]:
q_prescriptions_med = """
    SELECT pr.subject_id, pr.hadm_id, pr.pharmacy_id, pr.drug, pr.drug_type, pr.gsn, pr.ndc, pr.formulary_drug_cd
    FROM 
        mimic_hosp.prescriptions pr
        INNER JOIN mimic_core.patients pat 
            ON pr.subject_id = pat.subject_id
    WHERE anchor_year_group = '2017 - 2019'
"""
df_prescriptions_med = pd.read_sql_query(q_prescriptions_med, db_conn)

In [ ]:
df_prescriptions_med

In [ ]:
total_prescription_med = len(df_prescriptions_med)
present_drug = len(df_prescriptions_med[df_prescriptions_med['drug'].notna()])
present_gsn = len(df_prescriptions_med[df_prescriptions_med['gsn'].notna() & (df_prescriptions_med['gsn'] != '')])
present_ndc = len(df_prescriptions_med[(df_prescriptions_med['ndc'].notna()) & (df_prescriptions_med['ndc'] != '0')])
present_formulary = len(df_prescriptions_med[df_prescriptions_med['formulary_drug_cd'].notna()])

In [ ]:
print(f'{present_drug/total_prescription_med:.2%} drug names present in prescriptions')
print(f'{present_gsn/total_prescription_med:.2%} GSN codes present in prescriptions')
print(f'{present_ndc/total_prescription_med:.2%} NDC codes present in prescriptions') 
print(f'{present_formulary/total_prescription_med:.2%} formulary drug codes present in prescriptions')


In [ ]:
# check for how many are remaining
pr_medication_gsn_no_ndc = ((df_prescriptions_med['ndc'].isna() | (df_prescriptions_med['ndc'] == '0')) &  df_prescriptions_med.gsn.notna()).sum()
pr_medication_fdc_no_gsn_or_ndc = (((df_prescriptions_med['ndc'].isna() | (df_prescriptions_med['ndc'] == '0')) 
                                    & df_prescriptions_med.gsn.isna()) & df_prescriptions_med['formulary_drug_cd'].notna()).sum()

print(f'{pr_medication_gsn_no_ndc/total_prescription_med:.2%} gsn drug code present when ndc is not')
print(f'{pr_medication_fdc_no_gsn_or_ndc/total_prescription_med:.2%} formulary drug codes drug present when ndc/gsn are not')

#### Look for same med identifier with different secondary identifiers

In [ ]:
df_prescriptions_med.groupby(df_prescriptions_med['drug'])